In [1]:
# Setup
import sys
sys.path.append('/Users/taube/Classes/mus499mrc/Sources')
import musx
musx.setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf2")
%pwd

'/Users/rochelle/mus305/rtham3/synthesis'

# Synthesis and Spectral Composition

* In sound synthesis complex waves are generated by algorithms of some type:

    * Ring Modulation
    * Frequency Modulation
    * Amplitude Modulation
    * Karplus Strong
    * Granular Synthesis
    * Physical Modeling

Spectral composers adapt these synthesis techniques to compute 'symbolic spectra' (pitch sets) rather than, or in addition to, audio signals.

## Ring Modulation

In ring modulation two signals f1 and f2 are multiplied, which results in *sum and difference* tones: f1+f2 and |f1-f2|. 

<img src="img/rm1.png" width=500 />

If f1 and f2 are sine waves then the output spectrum consists of two sidebands, but if either f1 or f2 is a complex wave the output spectrum consists of the sum and difference tones for each harmonic in both signals.

This results in an output signal with the following characteristics:

* The output spectrum is not related to the inputs by a harmonic relationship (partials are additive not multiplicative)

* f1 and f2 do not appear in resulting spectrum.

* Each partial in f1 produces the sum and difference with all partials in f2, which often results in often dense sound.


`mux.rmspectrum(reqs1, freqs2, asfreqs=False)`

In musx the function mux.rmspectrum() returns the ring-modulated spectrum of two input signals, either of which can be a frequency, a list of frequencies, or a musx.Spectrum of frequencies.  The output spectrum is returned as a Spectrum object or a list of frequncues if *asfreqs* is true.

## Example Spectra

This example plays an input spectra followed by its ring modulated result. The first half note in each measure is the input specta performed as notes and the second half note is the ring-modulated output spectrum performed as notes. The first input signal is always C5, the second input signal contains the rest of the notes in each list.

<img src="img/rm2.png" width=700 />

In [ ]:
def rmchords(score, sets, dur):
    for set in sets:
        res = musx.rmspectrum(set[0], set[1:])
        for k in set:
            m = musx.Note(time=score.now, pitch=musx.keynum(k), duration=dur)
            score.add(m)
        # print(musx.pitch(set, hz=True), " -> ", musx.pitch(res.keynums()))
        for k in res.keynums():
            m = musx.Note(time=score.now + dur, pitch=k, duration=dur)
            score.add(m)
        yield dur*2

inputs = musx.hertz(["c5 b", "c5 bf", "c5 a", "c5 af", "c5 g",
                    "c5 fs", "c5 f", "c5 e", "c5 ef", "c5 d", "c5 db",
                    "c5 ef b", "c5 f bf", "c5 f b", "c5 fs b",
                    "c5 af b", "c5 f fs b", "c5 d f g a"])   
print(inputs)

In [ ]:
s1 = musx.MidiFile.metatrack()
s2 = musx.Seq()
score = musx.Score(out=s2)
score.compose(rmchords(score, inputs, .75))
f = musx.MidiFile("rm.mid", [s1,s2]).write()
musx.playfile(f.pathname)

## Example: ring modulation  etude

In [ ]:
def accompaniment(score, reps, dur, set1, set2):
    """Creates the accompanyment part from the ring modulation input specta."""
    # Create a cycle of the two inputs
    pat = musx.cycle([set1, set2])
    for _ in range(reps*2):
        # Get the next set.
        keys = next(pat)
        # Iterate the keys, play as a chord.
        for k in keys:
            # Create a midi note at the current time.
            m = musx.Note(time=score.now, duration=dur, pitch=k, amplitude=.3, instrument=0)
            # Add it to our output seq.
            score.add(m)
        # Wait till the next chord.
        yield dur
        
print('OK!')

In [ ]:
def melody(score, reps, dur, set3):
    """Creates the melodic part out of the ring modulated output specta."""
    # Create a cycle of the two inputs
    pat = musx.cycle(set3)
    for _ in range(2 * reps):
        m = musx.Note(time=score.now, duration=dur/2, pitch=next(pat), amplitude=.7, instrument=1)
        score.add(m)
        # Wait till the next note
        yield dur

print('OK!')

In [ ]:
from random import shuffle

def rmfunky(score, reps, dur, keys):
    """
    Main composer chooses input spectra , creates a ring modulated
    output spectrum and plays them using two parts.
    """
    num = musx.choose([1,2,3])
    # scramble the cycle of fourths
    pat = musx.jumble(keys)
    for _ in range(reps):
        # input1 is 1, 2 or 3 notes from cycle of 4ths
        keys1 = [next(pat) for _ in range(next(num))]
        # input2 is same
        keys2 = [next(pat) for _ in range(next(num))]
        # ring modulate the inputs
        spect = musx.rmspectrum([musx.hertz(k) for k in keys1], [musx.hertz(k) for k in keys2])
        # convert to keynums
        keys3 = spect.keynums(quant=1, unique=True, minkey=21, maxkey=108)
        # sprout composers to play inputs and output
        playn = musx.pick(3,4,5)
        score.compose(accompaniment(score, playn, dur, keys1, keys2))
        shuffle(keys3)
        print(keys2)
        score.compose(melody(score, playn, dur, keys3))
        # do it again after composers finish
        yield (dur * playn * 2)

print('OK!')

In [ ]:
from musx.midi.gm import Marimba, Clarinet
t0 = musx.MidiFile.metatrack(ins={0: Marimba, 1: Clarinet})
t1 = musx.Seq()
score = musx.Score(t1)
keys = musx.scale(40, 12, 5)
rhy = musx.intempo(.25, 74)
score.compose(rmfunky(score, 24, rhy, keys))
f = musx.MidiFile("rm.mid", [t0, t1]).write()
musx.playfile(f.pathname)

## Frequency Modulation

In Frequency Modulation (FM), the frequency of a carrier wave is altered by a modulator signal:

<img src="img/fm1.png" width=300/>   <img src="img/fm2.png" width=300 />

## Deviation and Sidebands

Deviations of the carrier (c) causes spectral sidebands to appear at multiples of the modulator (m) above and below the carrier:

<b>c ± (k*m)  where k=sideband order  0,1,2...</b>

<img src="img/fm3.png" width=500 />

## The C:M Ratio

* The carrier to modulator ratio (C:M) determines the harmonicity of the resulting spectrum.

* For harmonic spectra, M must have an integer relationship to C

* Non-integerratiosproduceinharmonicspectra

<img src="img/fm4.png" width=500 />

## FM Index

* The FM index (I) controls the amount of frequency deviation around the carrier: dev=m* I

* The higher the index the more sidebands are active: k=round(I)+1;  spectral density=2k+1

<img src="img/fm5.png" width=800 />

## Composing with FM

FM is a powerful algorithm for generating spectral note sets!

* Can generate inharmonic (dissonant) or harmonic (consonant) sets.

* Requires only three basic parameters:
    * carrier (center frequency)
    * c/m ratio (carrier/modulator ratio)
    * index (density, or width, of the spectrum.)

```fmspectrum(carrier, ratio, index)```

musx provides the `fmspectrum` function that returns an FM generated Spectrum given a carrier (in hertz), a carrier to modulator ratio, and an FM index.

**NOTE: fmspectrum requires the scipy package:**

```    (venv) $ python -m pip install scipy```

In [ ]:
fmspec = musx.fmspectrum(400, 1/4, 8)
print(fmspec.freqs())

In [ ]:
from math import e

fmspec = musx.fmspectrum(100, e, 3)
print(fmspec.freqs())

Convert to floating point key numbers:

In [ ]:
print(fmspec.keynums())

Convert to pitches or pitch classes, etc.:

In [ ]:
print(musx.pitch(fmspec.keynums(minkey=36, maxkey=84, unique=True)))

print([round(k) % 12 for k in fmspec.keynums(minkey=36, maxkey=84, unique=True)])

## Example

Generate FM chords with random fluctuations of cm ratio and index:

In [ ]:
def fmchords(score, cen, cm1, cm2, in1, in2, rhy, tune):
    for c in cen:
        carrier = musx.hertz(c)
        cmrat = musx.between(cm1, cm2)
        index = musx.between(in1, in2)
        spectrum = musx.fmspectrum(carrier, cmrat, index)
        #print("spectrum freqs:", spectrum.freqs())
        keynums = spectrum.keynums(minkey=c-12, maxkey=c+12)
        #print("spectrum keynums:", keynums)
        for k in keynums:
            note = musx.Note(time=score.now, pitch=k, duration=rhy)
            score.add(note)
        yield rhy

In [ ]:
s1 = musx.MidiFile.metatrack(microdivs=1)
s2 = musx.Seq()
score = musx.Score(out=s2)
m = musx.cycle([i for i in range(64, 80, 2)], stop=16)
score.compose(fmchords(score, m, 1.0, 2.0, 2.0, 3.0, 2, 4))
#score.compose(fmchords(score, 5, 60, 1.0, 2.0, 2.0, 4.0, 1, 1))
f = musx.MidiFile("fm.mid", [s1,s2]).write()
musx.playfile(f.pathname)

## Example: FM improvisor

An FM improvisor where specra produce melodic gestures 70% of the time otherwise they are performed as chords. The improvisation is controlled by a melodic line that provides the carrier for each of the computed specta.

The melodic contour:

In [ ]:
contour = musx.keynum("a4 g f e a4 b c d gs b c5 ef fs gs " 
                      "a5 bf g f e a5 b c d gs3 f e cs c " 
                      "bf5 gs5 as3 cs5 e6 f4 gs5 d6 e f g "
                      "c5 b a g bf c5 cs e4 f gs d4 c b "
                      "a4 e5 f g a5")

fm_improv() uses the contour line as a series of carrier frequencies (specified as midi keynums) to produces fm spectra that is converted into both melodic and harmonic gestures. 

In [ ]:
import random

def fmimprov(score, line, beat):
    amp = .7
    dur = beat
    for knum in line:
        ismel = musx.odds(.7)
        rhy = musx.pick(dur, dur / 2, dur / 4)
        f, r, x = musx.hertz(knum), musx.between(1.1, 1.9), musx.pick(1, 2, 3)
        print("\ncarrier=",f,"c/m ratio=",r,"fm index=",x)
        spec = musx.fmspectrum(f,r,x)
        keys = spec.keynums(unique=True, minkey=knum-14, maxkey=knum+14)
        if ismel:
            random.shuffle(keys)
        sub = rhy / len(keys) if ismel else 0
        #print("melody:" if ismel else "chord:", "time=", q.now, "dur=", rhy, "keys=", keys)
        for i, k in enumerate(keys):
            m = musx.Note(time=score.now + (i * sub), duration=dur, pitch=k, amplitude=amp)
            score.add(m)
        yield rhy

In [ ]:
s1 = musx.MidiFile.metatrack(microdivs=1)
s2 = musx.Seq()
score = musx.Score(out=s2)
score.compose(fmimprov(score, contour, 1))
#q.compose(fmchords(q, 5, 60, 1.0, 2.0, 2.0, 4.0, 1, 1))
f = musx.MidiFile("fm.mid", [s1,s2]).write()
musx.playfile(f.pathname)

## Example: The Aeolian Harp

Section inharmonic specra, equal-temperment vs just FM (tape), "exponential" rhythms.

In [ ]:
# page 12 / 14 / 16
!open ../../Website/downloads/aeolian-harp/ah.pdf

In [ ]:
# 10:55/ 13:00 / 13:30
!open ../../Website/downloads/aeolian-harp/ah.mp3

## Example: Le Tombeau de Harvey

By Dongryul Lee (DMA 2020 UIUC, now post-doc at university of Chicago. An FM piano piece won 3rd place in international Bartok Piano Composition Contest

(Jonathan Harvey's famous piece [Tombeau De Messiaen](https://www.youtube.com/watch?v=yKilziiMufQ) used FM tape against a live equal tempered piano)

[Short presentation](https://youtu.be/rWtjVZ9czNI?t=242)

[Performance](https://youtu.be/UDu8VvuDwqs?t=79)

In [ ]:
!open DongryulLEE_LeTombeauDeHarvey_17-08-2020.pdf

## In class exercise

Use interpolation with fmspectrum to move from a "consonant" sounding spectrum to a "noisy" spectrum and then back to consonance.